<a href="https://colab.research.google.com/github/Sejong-Kaggle-Challengers/jeongmin/blob/main/KCB_%EA%B8%88%EC%9C%B5%EC%8A%A4%ED%83%80%EC%9D%BC_%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### KCB(올크레딧) - 신용평가사
다양한 금융사로부터 여신거래 기록, 채무 정보, 연체정보 등  다양한 고객 금융 정보를 받아 전국민 85.4%인 4,400만 경제활동 인구의
경제 활동 데이터를 보유하고 있으며, 이를 통해 개인 신용 등급 책정 및 솔루션/컨설팅 서비스를 제공
<br><br>
대회 목적 :
1. KCB 전국민 금융 데이터의 새로운 인사이트 발굴
2. 활용도 높은 결과물로 금융친화적 콘텐츠 발전에 기여
<br><br>
credit_card_data.csv : 전국민 카드 및 대출 이용통계 데이터
연령대/거주지/성별 구분에 따른 카드 및 대출 이용통계 데이터로, 구분 값 별 카드이용금액 비교, 대출금액 및 대출 특이사항 비교 등이 가능하다.

In [1]:
!unzip data08.zip

Archive:  data08.zip
  inflating: KCB 대회 약관.pdf  
  inflating: credit_card_data.csv    
  inflating: jeju_financial_life_data.csv  


In [11]:
!pip install eli5

     |████████████████████████████████| 112kB 5.4MB/s 


In [13]:
!pip install shap

     |████████████████████████████████| 358kB 5.7MB/s 
  Created wheel for shap: filename=shap-0.38.1-cp36-cp36m-linux_x86_64.whl size=489404 sha256=be0e193532db602e1eb7842f70e2a1f6312c4d770b32bf4aca4d39790337cc25
  Stored in directory: /root/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap


In [37]:
import pandas as pd #분석
import matplotlib.pyplot as plt #시각화
import seaborn as sns #시각화
import numpy as np#분석

#경고메세지 무시
import warnings
warnings.filterwarnings('ignore')

#%matplotlib inline --> Rich output 를 바로 볼 수 있도록 (colab 에서는 굳이 쓰지 않아도 되는 것 같다)
color = sns.color_palette() #팔레트

#모델
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

#Permutation Importance를 통해 변수의 중요도를 알아볼 수 있다.
import eli5
from eli5.sklearn import PermutationImportance

#Shap Value는 실제값과 예측치의 차이를 설명하는 것
import shap

In [38]:
plt.rcParams["figure.facecolor"] = 'w' # background color = white
plt.rcParams["font.family"] = 'NanumBarunGothic' #글꼴 설정 = 나눔고딕
plt.rcParams['axes.unicode_minus'] = False #그래프에서 마이너스 기호가 깨지지 않고 표시되도록

In [49]:
train = pd.read_csv("credit_card_data.csv")
train.shape

(3888, 26)

In [40]:
train.head()

,pop_cd,year,month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment
0,L011,2016,1,812,2,서울,NaN,10대,55000,1.44,1.13,70000,92630000,87350000,85790000,0,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0
1,L011,2016,2,773,4,서울,NaN,10대,380000,1.71,0.96,170000,31140000,28910000,25850000,190000,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000
2,L011,2016,3,771,4,서울,NaN,10대,387000,1.75,0.95,190000,30230000,28180000,25110000,170000,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000
3,L011,2016,4,770,4,서울,NaN,10대,412000,1.76,0.92,200000,30430000,27710000,24630000,190000,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000
4,L011,2016,5,771,4,서울,NaN,10대,414000,1.75,0.94,170000,31210000,27620000,24390000,210000,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000


Nan으로 표현된 부분
- 지역별 통계 : city가 값이 있는 경우는 sex가 Nan --> sex = 전체
- 성별 통계 : sex가 값이 있는 경우는 city가 Nan --> city = 전국



In [50]:
#city와 sex에서 Nan값
train.isnull().sum()

pop_cd                                 0
year                                   0
month                                  0
avg_score                              0
avg_rat                                0
city                                 432
sex                                 3456
ages                                   0
population                             0
num_opencard                           0
num_usecard                            0
monthly_card_spend                     0
monthly_lc                             0
monthly_loan                           0
monthly_bk_loan                        0
monthly_cd_loan                        0
monthly_installments_loan              0
monthly_insurance_loan                 0
monthly_sbk_loan                       0
loan_commitment                        0
inst_rep_loanb                         0
ls_rep_loanb                           0
credit_loan                            0
mortgage_loan                          0
credit_card_paym

In [51]:
train['city'] = train['city'].fillna('전국')
train['sex'] = train['sex'].fillna('전체')

- 나이별 통계 --> 연산의 용이성을 위해 숫자로 변환
- 연도, 월 데이터는 python의 datatime 포맷으로 변환하여 새로운 열 year_month로 변환

In [52]:
train['ages'][0][:-1]

'10'

In [54]:
train['ages'] = train['ages'].apply(lambda x:int(x[:-1])).astype(float)
train['year_month'] = pd.to_datetime((train.year*100+train.month).apply(str), format = '%Y%m')
train.drop(['year','month'], axis=1, inplace=True) #원래 year, month 행 drop
#year_month 행을 이전의 year과 month 자리인 행 2번째로 자리를 변경해 train을 완성한다.
train = train[[train.columns[0], 'year_month'] + list(train.columns[1:-1])]

In [55]:
train.head()

,pop_cd,year_month,avg_score,avg_rat,city,sex,ages,population,num_opencard,num_usecard,monthly_card_spend,monthly_lc,monthly_loan,monthly_bk_loan,monthly_cd_loan,monthly_installments_loan,monthly_insurance_loan,monthly_sbk_loan,loan_commitment,inst_rep_loanb,ls_rep_loanb,credit_loan,mortgage_loan,credit_card_payment,credit_card_installments_payment
0,L011,2016-01-01,812,2,서울,전체,10.0,55000,1.44,1.13,70000,92630000,87350000,85790000,0,0,0,0,1120000,8210000,78020000,92170000,87260000,0,0
1,L011,2016-02-01,773,4,서울,전체,10.0,380000,1.71,0.96,170000,31140000,28910000,25850000,190000,510000,220000,1020000,680000,4660000,23300000,8260000,42410000,4000,1000
2,L011,2016-03-01,771,4,서울,전체,10.0,387000,1.75,0.95,190000,30230000,28180000,25110000,170000,490000,210000,1140000,640000,4370000,22930000,8300000,41710000,3000,1000
3,L011,2016-04-01,770,4,서울,전체,10.0,412000,1.76,0.92,200000,30430000,27710000,24630000,190000,510000,240000,1340000,510000,4770000,22160000,9460000,41040000,4000,1000
4,L011,2016-05-01,771,4,서울,전체,10.0,414000,1.75,0.94,170000,31210000,27620000,24390000,210000,490000,340000,1490000,640000,4740000,21960000,9660000,40850000,3000,1000
